In [1]:
import numpy as np
import string 


punctuations = list(string.punctuation) + ['।']
print(punctuations)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '।']


In [2]:
def load_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.strip().split(' _ _ ') for line in f.readlines()]
        # return data
        # empty lines are used to separate sentences
        # separate them into sentences
        sentences = []
        cur = []
        for line in data:
            if line == ['']:
                cur = [tuple(line) for line in cur]
                sentences.append(cur)
                cur = []
            else:
                for p in punctuations:
                    line[0] = line[0].replace(p, '')
                if len(line[0]) == 0:
                    line[0] = ' '
                cur.append(line)
        # # convert each list to a tuple
        sentences.append(cur)
        return sentences

In [3]:
train = load_data('../data/train.txt')

In [4]:
dev = load_data('../data/dev.txt')

In [6]:
!pip install bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 3.6 MB/s eta 0:00:0000:0100:01


In [7]:
!git clone https://github.com/sagorbrur/bnlp

Cloning into 'bnlp'...
remote: Enumerating objects: 1453, done.
remote: Counting objects: 100% (656/656), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 1453 (delta 381), reused 582 (delta 331), pack-reused 797
Receiving objects: 100% (1453/1453), 22.63 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (836/836), done.


In [9]:
from bnlp import POS
bn_pos = POS()
model_path = "bnlp/model/bn_pos.pkl"
text = "আমি ভাত খাই।" # or you can pass ['আমি', 'ভাত', 'খাই', '।']
res = bn_pos.tag(model_path, text)
print(res)


[('আমি', 'PPR'), ('ভাত', 'NC'), ('খাই', 'VM'), ('।', 'PU')]


In [10]:
def add_pos(sentence):
    # first one of the tuple is the word and the second one is the ner  
    words = []
    for word in sentence:
        words.append(word[0])
    pos = bn_pos.tag(model_path, words)
    ## add ner back to the tuple
    ret = []
    for i in range(len(pos)):
        word = sentence[i][0]
        ner = sentence[i][1]
        ret.append((word, pos[i][1], ner))
    return ret

def add_pos_to_all(sents):
    ret = []
    for i in range(len(sents)):
        if i % 100 == 0:
            print(i)
        ret.append(add_pos(sents[i]))
    return ret

In [11]:
add_pos(train[0])

[('তার', 'PPR', 'O'),
 ('মৃত্যুর', 'NC', 'O'),
 ('দশ', 'JQ', 'O'),
 ('দিন', 'NC', 'O'),
 ('পর', 'NST', 'O'),
 ('১১৫', 'RDF', 'O'),
 ('কৃষ্ণাঙ্গ', 'NC', 'O'),
 ('উচ্চ', 'JJ', 'O'),
 ('বিদ্যালয়ের', 'NC', 'O'),
 ('শিক্ষার্থীরা', 'NC', 'O'),
 ('তার', 'PPR', 'O'),
 ('হত্যার', 'NC', 'O'),
 ('প্রতিবাদে', 'NC', 'O'),
 ('ম্যাককম্ব', 'NC', 'B-LOC'),
 ('এর', 'PPR', 'O'),
 ('মাধ্যমে', 'PP', 'O'),
 ('মিছিল', 'NC', 'O'),
 ('করেছে', 'VM', 'O')]

In [12]:
train_sents =  add_pos_to_all(train)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200


In [13]:
dev_sents = add_pos_to_all(dev)

0
100
200
300
400
500
600
700


In [14]:
def get_all_words(sents):
    words = []
    for i in range(len(sents)):
        for w in sents[i]:
            words += [w[0]]
    return words

In [15]:
all_train_words = get_all_words(train_sents)
len(all_train_words)

191897

In [16]:
word_freq = {}
for w in all_train_words:
    if w in word_freq:
        word_freq[w] += 1
    else:
        word_freq[w] = 1

In [17]:
## feature extraction for Conditional Random Field - Bangla NER
## feature extraction for Conditional Random Field - Bangla NER

def wordToFeatures(sent, idx):
    word = sent[idx][0]
    postag = sent[idx][1]
    
    features = {
        'bias': 1.0,
        'word': word,
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word.isdigit': word.isdigit(),
        'index': idx,
        'length': len(word),
        'postag': postag,
        'freq': word_freq[word] if word in word_freq else 0,
    }
        
    
    for i in range(1, 3):
        if idx < i:
            break
        wordi = sent[idx-i][0]
        postagi = sent[idx-i][1]
        features.update({
            '-{}:word'.format(i): wordi,
            '-{}:word[-3:]'.format(i): wordi[-3:],
            '-{}:word[-2:]'.format(i): wordi[-2:],
            '-{}:word[:3]'.format(i): wordi[:3],
            '-{}:word[:2]'.format(i): wordi[:2],
            '-{}:word.isdigit'.format(i): wordi.isdigit(),
            '-{}:postag'.format(i): postagi,
        })
    
    for i in range(1, 3):
        if (idx+i) >= len(sent):
            break
        wordi = sent[idx+i][0]
        postagi = sent[idx+i][1]
        features.update({
            '{}:word'.format(i): wordi,
            '{}:word[-3:]'.format(i): wordi[-3:],
            '{}:word[-2:]'.format(i): wordi[-2:],
            '{}:word[:3]'.format(i): wordi[:3],
            '{}:word[:2]'.format(i): wordi[:2],
            '{}:word.isdigit'.format(i): wordi.isdigit(),
            '{}:postag'.format(i): postagi,
        })
        
    if idx == 0:
        features['BOS'] = True
    if idx == len(sent) - 1:
        features['EOS'] = True
    
    return features

def sentTofeatures(sent):
    return [wordToFeatures(sent, i) for i in range(len(sent))]

def sentTolabels(sent):
    return [label for token, postag, label in sent]

In [18]:
train_sents[0]

[('তার', 'PPR', 'O'),
 ('মৃত্যুর', 'NC', 'O'),
 ('দশ', 'JQ', 'O'),
 ('দিন', 'NC', 'O'),
 ('পর', 'NST', 'O'),
 ('১১৫', 'RDF', 'O'),
 ('কৃষ্ণাঙ্গ', 'NC', 'O'),
 ('উচ্চ', 'JJ', 'O'),
 ('বিদ্যালয়ের', 'NC', 'O'),
 ('শিক্ষার্থীরা', 'NC', 'O'),
 ('তার', 'PPR', 'O'),
 ('হত্যার', 'NC', 'O'),
 ('প্রতিবাদে', 'NC', 'O'),
 ('ম্যাককম্ব', 'NC', 'B-LOC'),
 ('এর', 'PPR', 'O'),
 ('মাধ্যমে', 'PP', 'O'),
 ('মিছিল', 'NC', 'O'),
 ('করেছে', 'VM', 'O')]

In [19]:
%%time
X_train = [sentTofeatures(s) for s in train_sents]
y_train = [sentTolabels(s) for s in train_sents]

CPU times: user 3.26 s, sys: 698 ms, total: 3.96 s
Wall time: 3.97 s


In [20]:
%%time
X_dev = [sentTofeatures(s) for s in dev_sents]
y_dev = [sentTolabels(s) for s in dev_sents]

CPU times: user 169 ms, sys: 34.8 ms, total: 204 ms
Wall time: 202 ms


In [22]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics

In [23]:
crf = crfsuite.CRF(
    verbose='true',
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=150,
    all_possible_transitions=True
)
try:
    crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
except:
    pass

loading training data to CRFsuite: 100%|██████████| 15300/15300 [00:09<00:00, 1612.74it/s]


loading dev data to CRFsuite: 100%|██████████| 800/800 [00:00<00:00, 1535.51it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 357323
Seconds required: 2.647

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 150
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=8.39  loss=383191.34 active=351278 precision=0.064  recall=0.077  F1=0.070  Acc(item/seq)=0.838 0.000  feature_norm=0.12
Iter 2   time=1.72  loss=380107.47 active=342106 precision=0.064  recall=0.077  F1=0.070  Acc(item/seq)=0.838 0.000  feature_norm=0.12
Iter 3   time=1.73  loss=186812.46 active=295193 precision=0.064  recall=0.077  F1=0.070  Acc(item/seq)=0.838 0.000  feature_norm=0.32
Iter 4   time=6.73  loss=173067.87 active=336675 precision=0.064  recall=0.077  F1=0.070  Acc(item/seq)=0.838 0.000  feature_norm=0.35
Iter 5   time=1.77  loss=171825.77 activ

In [24]:
# %%time
# crf = crfsuite.CRF(
#     algorithm='lbfgs',
#     c1=0.1,
#     c2=0.1,
#     max_iterations=100,
#     all_possible_transitions=True
# )
# crf.fit(X_train, y_train)

In [25]:
labels = list(crf.classes_)
labels

['O',
 'B-LOC',
 'B-GRP',
 'I-GRP',
 'B-PROD',
 'B-CW',
 'I-CW',
 'B-CORP',
 'B-PER',
 'I-PER',
 'I-CORP',
 'I-PROD',
 'I-LOC']

In [26]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='macro', labels=labels)

0.6817459498430333

In [36]:
def load_test(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.strip() for line in f.readlines()]
        
        sentences = []
        cur = []
        for i, line in enumerate(data):
            if line == '':
                cur = [tuple([line, ' ']) for line in cur]
                sentences.append(cur)
                cur = []
            else:
                for p in punctuations:
                    line = line.replace(p, '')
                if len(line) == 0:
                    line = ' '
                cur.append(line)
        # # convert each list to a tuple
        sentences.append(cur)
        return sentences

In [37]:
test = load_test('../data/test.txt')
len(test)

13215

In [39]:
test_sents =  add_pos_to_all(test)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200


In [43]:
X_test = [sentTofeatures(s) for s in test_sents]

In [46]:
len(X_test)

13215

In [44]:
y_pred = crf.predict(X_test)

In [47]:
with open("predictions_crf_baseline.txt", "w") as file:
    for i, p in enumerate(y_pred):
        if i > 0:
            file.write('\n')
        for l in p:
            file.write(l +"\n")
        